# New Section

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
!pip install underthesea
!pip install gensim
!pip install fasttext

     |████████████████████████████████| 7.5MB 10.2MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 6.7MB 43.6MB/s 
     |████████████████████████████████| 245kB 48.3MB/s 
     |████████████████████████████████| 747kB 46.8MB/s 
     |████████████████████████████████| 753.2MB 23kB/s 
     |████████████████████████████████| 1.3MB 45.0MB/s 
     |████████████████████████████████| 890kB 46.6MB/s 
     |████████████████████████████████| 1.1MB 49.7MB/s 
     |████████████████████████████████| 2.9MB 37.0MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=95d076be957d99174944d8c88105921191a3cec069a6cad8cd8977ca477ac066
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=54aea08373db7f6b104099a880156a68648aebe1ed3422cf22dc1f2b34883c80
  Stored in directory

In [ ]:
import os
import re
import gzip
import keras
import fasttext
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import string
from underthesea import word_tokenize
from gensim.utils import simple_preprocess
from gensim.models.wrappers import FastText
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential 
from keras.layers import Dense, Embedding 
from keras.layers import LSTM 
from keras.layers.wrappers import Bidirectional
from keras.layers import Dense, Lambda, dot, Activation, concatenate, Embedding, LSTM, Dropout, Conv1D, MaxPooling1D, Flatten, BatchNormalization, AveragePooling1D
import gensim as gs
import tensorflow as tf

from gensim.models import FastText

In [ ]:
def getAllpath(pathContent):
  for dirname, _, filenames in os.walk(pathContent):
    for filename in filenames:
        print(os.path.join(dirname, filename))
pathContent='/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/'
getAllpath(pathContent)

/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/Vietnamese_stopwords/vietnamese-stopwords-dash.txt
/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/datacsv/1sao.csv
/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/datacsv/2sao.csv
/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/datacsv/3sao.csv
/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/datacsv/4sao.csv
/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/embeddings/cc.vi.300.bin
/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/embeddings/w2vsg_embdeding.model
/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/model_predict/model128.h5


Create stopword list


In [ ]:
def create_stopwords(path,original_stopwords):
  with open(path, encoding="utf-8") as words:
    return [w[:len(w) - 1] for w in words] + original_stopwords

original_stopwords = ["tiki","lazada", "shopee"]
stopwords = create_stopwords('/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/Vietnamese_stopwords/vietnamese-stopwords-dash.txt',original_stopwords)


In [ ]:
len(stopwords)

1867

Get data


In [ ]:
names = ["classes", "content"]
data1sao=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/datacsv/1sao.csv', names=names)
data2sao=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/datacsv/2sao.csv', names=names)
data3sao=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/datacsv/3sao.csv', names=names)
data4sao=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/datacsv/4sao.csv', names=names)
dataset=pd.concat([data1sao,data2sao,data3sao,data4sao])

In [ ]:
dataset.shape

(36844, 2)

In [ ]:
def clean_text(text):  
    # preprocessing ....
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.translate(str.maketrans('', '', string.digits))
    ## Remove puncuation
    text = text.translate(string.punctuation)
    ## Convert words to lower case and split them
    text = text.lower().split()
    text = " ".join(text)
    return text
dataset['content'] = dataset['content'].map(lambda x: clean_text(x))

In [ ]:
def word_processing_teacher(sentence,stopwords):
  sentence = " ".join(simple_preprocess(sentence))
  sentence = [word for word in word_tokenize(sentence.lower(), format="text").split() if word not in stopwords]
  return [word for word in sentence if word != ""]
input_sentences = [word_processing_teacher(str(sentence),stopwords) for sentence in dataset["content"].values.tolist()]

In [ ]:
data_label_1 = dataset[dataset['classes']==1]['content'].values
data_label_2 = dataset[dataset['classes']==2]['content'].values
data_label_3 = dataset[dataset['classes']==3]['content'].values
data_label_4 = dataset[dataset['classes']==4]['content'].values
print(data_label_1[:5])

['vỏ ốp tai nghe không có đèn báo hiệu chất lượng âm thanh kém tín hiệu chập chờn'
 'bị treo khi khởi động lại bằng app tether phải khởi động thủ công bằng tay tắt và mở lại nguồn khởi động hơn phút khá là lâu so với các thiết bị router tầm trung khác kiểm tra cùng các hãng technicolor tplink wrn mặc dù quảng cáo là chuẩn n max mb nhưng chất lượng chậm còn thua các router chuẩn n max mb kiểm tra cùng đường truyền mb chắc không đạt chuẩn rồi băng tần g thì phát quá yếu cách m và qua lớp cửa gỗ thôi đã mất cột sóng đo bằng điện thoại samsung nên trong cùng tầm giá k nên chọn router ac băng tần kép có khi tận dụng được băng tần g lại ổn hơn hoặc chọn router chuẩn n max m là hợp lý'
 'chuột dùng rất ổn trong gần tháng đầu nhưng sau đó đột ngột không sử dụng được nữa đêm hôm trước dùng bt sáng hôm sau bật lên đèn nhưng chuột không di chuyển mình mua con ở tiki và ở shopee cách nhau tuần và con hỏng cách nhau đúng tuần có thể có vấn đề ở dòng sản phẩm này mình đã phải yêu cầu bảo hành từ tik

In [ ]:
data_train = dataset['content'].values
data_train.shape

(36844,)

In [ ]:
def tao_tu_dien(data):
  my_dict = {}
  for sentences in data:
    for words in sentences.split(' '):
      if words not in my_dict:
        my_dict[words] = 1
  return my_dict


In [ ]:
my_dict = tao_tu_dien(data_train)
print(my_dict)

{'vỏ': 1, 'ốp': 1, 'tai': 1, 'nghe': 1, 'không': 1, 'có': 1, 'đèn': 1, 'báo': 1, 'hiệu': 1, 'chất': 1, 'lượng': 1, 'âm': 1, 'thanh': 1, 'kém': 1, 'tín': 1, 'chập': 1, 'chờn': 1, 'bị': 1, 'treo': 1, 'khi': 1, 'khởi': 1, 'động': 1, 'lại': 1, 'bằng': 1, 'app': 1, 'tether': 1, 'phải': 1, 'thủ': 1, 'công': 1, 'tay': 1, 'tắt': 1, 'và': 1, 'mở': 1, 'nguồn': 1, 'hơn': 1, 'phút': 1, 'khá': 1, 'là': 1, 'lâu': 1, 'so': 1, 'với': 1, 'các': 1, 'thiết': 1, 'router': 1, 'tầm': 1, 'trung': 1, 'khác': 1, 'kiểm': 1, 'tra': 1, 'cùng': 1, 'hãng': 1, 'technicolor': 1, 'tplink': 1, 'wrn': 1, 'mặc': 1, 'dù': 1, 'quảng': 1, 'cáo': 1, 'chuẩn': 1, 'n': 1, 'max': 1, 'mb': 1, 'nhưng': 1, 'chậm': 1, 'còn': 1, 'thua': 1, 'đường': 1, 'truyền': 1, 'chắc': 1, 'đạt': 1, 'rồi': 1, 'băng': 1, 'tần': 1, 'g': 1, 'thì': 1, 'phát': 1, 'quá': 1, 'yếu': 1, 'cách': 1, 'm': 1, 'qua': 1, 'lớp': 1, 'cửa': 1, 'gỗ': 1, 'thôi': 1, 'đã': 1, 'mất': 1, 'cột': 1, 'sóng': 1, 'đo': 1, 'điện': 1, 'thoại': 1, 'samsung': 1, 'nên': 1, 'trong':

In [ ]:
my_dict_array = [sentence for sentence in my_dict]
my_dict_array

['vỏ',
 'ốp',
 'tai',
 'nghe',
 'không',
 'có',
 'đèn',
 'báo',
 'hiệu',
 'chất',
 'lượng',
 'âm',
 'thanh',
 'kém',
 'tín',
 'chập',
 'chờn',
 'bị',
 'treo',
 'khi',
 'khởi',
 'động',
 'lại',
 'bằng',
 'app',
 'tether',
 'phải',
 'thủ',
 'công',
 'tay',
 'tắt',
 'và',
 'mở',
 'nguồn',
 'hơn',
 'phút',
 'khá',
 'là',
 'lâu',
 'so',
 'với',
 'các',
 'thiết',
 'router',
 'tầm',
 'trung',
 'khác',
 'kiểm',
 'tra',
 'cùng',
 'hãng',
 'technicolor',
 'tplink',
 'wrn',
 'mặc',
 'dù',
 'quảng',
 'cáo',
 'chuẩn',
 'n',
 'max',
 'mb',
 'nhưng',
 'chậm',
 'còn',
 'thua',
 'đường',
 'truyền',
 'chắc',
 'đạt',
 'rồi',
 'băng',
 'tần',
 'g',
 'thì',
 'phát',
 'quá',
 'yếu',
 'cách',
 'm',
 'qua',
 'lớp',
 'cửa',
 'gỗ',
 'thôi',
 'đã',
 'mất',
 'cột',
 'sóng',
 'đo',
 'điện',
 'thoại',
 'samsung',
 'nên',
 'trong',
 'giá',
 'k',
 'chọn',
 'ac',
 'kép',
 'tận',
 'dụng',
 'được',
 'ổn',
 'hoặc',
 'hợp',
 'lý',
 'chuột',
 'dùng',
 'rất',
 'gần',
 'tháng',
 'đầu',
 'sau',
 'đó',
 'đột',
 'ngột',
 'sử',


In [ ]:
len_my_dict = len(my_dict)
len_my_dict

12950

In [ ]:
def tao_nhieu(sentence):
  so_nhieu = np.random.randint(3,6)
  for i in range(so_nhieu):
    left = np.random.randint(0,2)
    index_words = np.random.randint(0,len_my_dict)
    if left ==1 :
      sentence = my_dict_array[index_words] + ' ' + sentence
    else:
      sentence = sentence + ' ' + my_dict_array[index_words] 
  return sentence

In [ ]:
def reverse_sentences(sentence):
  sentence = sentence.split(' ')
  sentence.reverse()
  return ' '.join(x for x in sentence)

In [ ]:
def lanvung(sentence):
  sentence_arr = sentence.split(' ')
  len_sen = len(sentence_arr)
  if len_sen < 4:
    return sentence
  time_lan = np.random.randint(1,5)
  dict_time = {}
  for i in range(time_lan):
    while True:
      current_index = np.random.randint(0,len_sen)
      if current_index not in dict_time:
        dict_time[current_index] =1 
        sentence_arr[current_index] = (sentence_arr[current_index] + ' ' ) * np.random.randint(2,6)
        break
      else :
        continue
  return ' '.join(x for x in sentence_arr)

In [ ]:
data_nhieu =[ tao_nhieu(x) for x in data_label_1]
data_label_1 = np.concatenate((data_nhieu,data_label_1))
data_rever =[ reverse_sentences(x) for x in data_label_1]
data_label_1 = np.concatenate((data_rever,data_label_1))
data_loan =[ lanvung(x) for x in data_label_1]
data_label_1 = np.concatenate((data_label_1,data_loan))


In [ ]:
data_nhieu =[ tao_nhieu(x) for x in data_label_2]
data_label_2 = np.concatenate((data_nhieu,data_label_2))
data_rever =[ reverse_sentences(x) for x in data_label_2]
data_label_2 = np.concatenate((data_rever,data_label_2))
data_loan =[ lanvung(x) for x in data_label_2]
data_label_2 = np.concatenate((data_label_2,data_loan))

In [ ]:
data_nhieu =[ tao_nhieu(x) for x in data_label_3]
data_label_3 = np.concatenate((data_nhieu,data_label_3))
data_rever =[ reverse_sentences(x) for x in data_label_3]
data_label_3 = np.concatenate((data_rever,data_label_3))
data_loan =[ lanvung(x) for x in data_label_3]
data_label_3 = np.concatenate((data_label_3,data_loan))

In [ ]:
data_nhieu =[ tao_nhieu(x) for x in data_label_4]
data_label_4 = np.concatenate((data_nhieu,data_label_4))
data_rever =[ reverse_sentences(x) for x in data_label_4]
data_label_4 = np.concatenate((data_rever,data_label_4))
data_loan =[ lanvung(x) for x in data_label_4]
data_label_4 = np.concatenate((data_label_4,data_loan))

In [ ]:
label_1 = np.ones(len(data_label_1))*1
label_2 = np.ones(len(data_label_2))*2
label_3 = np.ones(len(data_label_3))*3
label_4 = np.ones(len(data_label_4))*4


In [ ]:
data_train = np.concatenate((data_label_1,data_label_2,data_label_3,data_label_4))

In [ ]:
labels =  np.concatenate((label_1,label_2,label_3,label_4))

In [ ]:
data_ft = np.concatenate((data_label_1[0:10000],data_label_2[0:10000],data_label_3[0:10000],data_label_4[0:10000]))
data_ft = [x.split() for x in data_ft ]

Create dict


In [ ]:
def generate_dict(datas):
  # tạo từ điển
  word2id = dict()
  max_words = 0 # maximum number of words in a sentence

  # Construction of word2id dict
  for sentence in datas:
      for word in sentence.split(' '):
          # Add words to word2id dict if not exist
          if word not in word2id:
              word2id[word] = len(word2id)
      # If length of the sentence is greater than max_words, update max_words
      if len(sentence) > max_words:
          max_words = len(sentence)
  return word2id, max_words
dict_and_maxWord=generate_dict(data_train)

In [ ]:
word2id=dict_and_maxWord[0]
maxWord=dict_and_maxWord[1]
lenWord=len(word2id)

In [ ]:
(maxWord,lenWord)


(2359, 12951)

In [ ]:
# Construction of label2id and id2label dicts
id2word = {v: k for k, v in word2id.items()}

In [ ]:
id2word.get(maxWord)

'nhượt'

Fix lenght

In [ ]:
# fix độ dài cho câu
X = [[word2id[word] for word in sentence.split(' ')] for sentence in data_train]
X = sequence.pad_sequences(X, maxWord)
Y = labels
le = LabelEncoder()
Y = le.fit_transform(Y)
#Y = Y.reshape(-1,1)
Y_train = keras.utils.to_categorical(Y, num_classes=4)

In [ ]:
(X.shape,Y_train.shape)

((294752, 2359), (294752, 4))

hôm qua tao ghi lộn :(((
  soryy phú
  đụ má nguyên cái embexding maxtrix ra 0 00 0 0 0  0 0 hết

In [ ]:
model_fasttext = FastText(size=350, window=15, min_count=3, workers=4, sg=1)
model_fasttext.build_vocab(data_ft)
model_fasttext.train(data_ft, total_examples = model_fasttext.corpus_count, epochs = model_fasttext.iter)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
word2id = dict({'':0})
max_words = 200

for sentence in data_train:
  s = sentence.split(' ')
  for word in s:
    if word not in word2id:
      word2id[word] = len(word2id)

id2word = {v: k for k, v in word2id.items()}

In [ ]:
X = [[word2id[word] for word in sentence.split(' ')] for sentence in data_train]
X_train = sequence.pad_sequences(X, max_words)

In [ ]:
# embedding matrix
print('preparing embedding matrix...')

MAX_NB_WORDS = 100000
words_not_found = []
embedding_dim = 350 # The dimension of word embeddings
nb_words = min(MAX_NB_WORDS, len(word2id))

# Với mỗi từ trong câu, lưu lại word vector phụ vụ để huấn luyện mô hình
embedding_matrix = np.zeros((nb_words, embedding_dim))
for word, i in word2id.items():
  try:
    if i >= nb_words:
      continue
    embedding_vector = model_fasttext[word]
    if (embedding_vector is not None) and len(embedding_vector) > 0:
      embedding_matrix[i] = embedding_vector

    else:
      words_not_found.append(word)
  except:
    words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))


preparing embedding matrix...
number of null word embeddings: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_train,Y_train,test_size=0.18)

**build model 1**

In [ ]:
# import tensorflow as tf
# from keras.initializers import Constant
# callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath="/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/model_predict/model128.h5", 
#     verbose=1, 
#     save_weights_only=True,
#     save_freq=5*32)

# model = Sequential()

# model.add(
#           Embedding(
#                       lenWord,
#                       embedding_dim, 
#                       embeddings_initializer=Constant(embedding_matrix),
#                       input_length=maxWord
#                     )
#         )
# model.add(Bidirectional(LSTM(128, return_sequences=True)))
# # # model.add(Dropout(0.5))x1
# # model.add(Dense(4, activation='softmax'))
# model.add(Dropout(0.2))
# model.add(LSTM(64))
# model.add(Dropout(0.5))

# model.add(Dense(4, activation="softmax"))

# model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])
# model.summary()
# # model1.save('/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/model_predict/model.h5')

In [ ]:
import tensorflow as tf
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


patience = 50
base_path='/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/model_predict/'
early_stop = EarlyStopping('val_loss', patience=patience)

reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1,
                              patience=int(patience/4), verbose=1)

trained_models_path = base_path + 'BI_LSTM_CNN'
model_names = trained_models_path + '.{epoch:02d}.hdf5'

model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,
                                   save_best_only=True)

callback = [model_checkpoint, early_stop, reduce_lr]

# callback = tf.keras.callbacks.ModelCheckpoint(
    # filepath="/content/drive/MyDrive/Colab Notebooks/Report2NLP/input/model_predict/model12899.hdf5", 
    # verbose=1, 
    # # save_weights_only=True,
    # save_freq="epoch")

model=Sequential()
model.add(Embedding(len(word2id),
    embedding_dim,
    embeddings_initializer=Constant(embedding_matrix),
    input_length=max_words))

model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(Dropout(0.5))

model.add(Conv1D(32,3,activation='relu'))
model.add(MaxPooling1D())

model.add(Conv1D(64,3,activation='relu'))
model.add(MaxPooling1D())

model.add(Conv1D(128,3,activation='relu'))
model.add(MaxPooling1D())

model.add(Flatten())
model.add(Dense(1024,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(4,activation='softmax'))

model.compile(loss = 'categorical_crossentropy',optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 200, 350)          4532850   
_________________________________________________________________
bidirectional_4 (Bidirection (None, 200, 1024)         3534848   
_________________________________________________________________
dropout_8 (Dropout)          (None, 200, 1024)         0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 198, 32)           98336     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 99, 32)            0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 97, 64)            6208      
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 48, 64)           

In [ ]:
train_history = model.fit(
    x_train, y_train,
    batch_size=1024,
    epochs=1,
    callbacks=callback,
    validation_data = (x_test, y_test)
)

237/237 [==============================] - ETA: 0s - loss: 0.8196 - accuracy: 0.6397
Epoch 00001: val_loss improved from inf to 0.70098, saving model to /content/drive/MyDrive/Colab Notebooks/Report2NLP/input/model_predict/BI_LSTM_CNN.01.hdf5
237/237 [==============================] - 455s 2s/step - loss: 0.8196 - accuracy: 0.6397 - val_loss: 0.7010 - val_accuracy: 0.7025


In [ ]:
plt.plot(train_history.history['accuracy'])
plt.plot(train_history.history['val_accuracy'])
plt.title('Train History')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()